In [ ]:
import os
import pickle
import time
from multiprocessing import Pool
from time import sleep

import networkx as nx
import numpy as np
import pandana as pdna
import pandas as pd
from tqdm.notebook import trange, tqdm
import igraph as ig
from scripts import graph_osm_loader, utils

Примеры как резолвить комьюнити

In [ ]:
# pdna.utils.

In [ ]:
#resolve_communities
def resolve_communities(H: nx.Graph, **params) -> list[set[int]]:
    communities = nx.community.louvain_communities(H,
                                                   seed=1534,
                                                   weight='length',
                                                   resolution=params['r'])
    return utils.validate_cms(H, communities)

In [ ]:
GRAPH_ID = 'R207359'  # R13470549 R2555133 R337422

In [ ]:
# примеры id есть в graph_osm_loader.py
g = graph_osm_loader.get_graph(GRAPH_ID)  # загрузка графа
len(g.nodes), len(g.edges)

In [ ]:
node2id = {u: i for i, u in enumerate(g.nodes())}
nodes_x = [d['x'] for _, d in g.nodes(data=True)]
nodes_y = [d['y'] for _, d in g.nodes(data=True)]
edges_from = [node2id[u] for u, _ in g.edges()]
edges_to = [node2id[v] for _, v in g.edges()]
edges_w = [d['length'] for u, v, d in g.edges(data=True)]
df = pd.DataFrame.from_dict({
    'x': nodes_x,
    'y': nodes_y
})
df1 = pd.DataFrame.from_dict(
    {
        'f': edges_from,
        't': edges_to,
        'w': edges_w
    }
)


In [ ]:
G = ig.Graph(n=len(nodes_x), edges=[[edges_from[i], edges_to[i]] for i in range(len(edges_to))],
                 edge_attrs={'weight': edges_w})

In [ ]:
net_ch = pdna.Network(df['x'], df['y'], df1['f'], df1['t'], df1[['w']])
# net_ch.precompute(10_000)

In [ ]:
path = utils.get_path('pouits', f'points_{GRAPH_ID}.pickle')

if os.path.exists(path):
    with open(path, 'rb') as fp:
        points = pickle.load(fp)
        fp.close()
else:
    points = [utils.get_node_for_initial_graph_v2(g) for _ in trange(1000, desc='generate points')]
    with open(path, 'wb') as fp:
        pickle.dump(points, fp)
        fp.close()

In [ ]:
del g

In [ ]:
NUM_ITERATION = 3  # чтобы уменьшить ошибку при вычислении времени выполнения, при каждом замере время меряется для NUM_ITERATION повторений
WORKER = 4  # количество потоков


def do_calc(data):
    pps, i = data

    stat = {}
    stat['l'] = []
    stat['h_l'] = []
    stat['p'] = []
    stat['h_p'] = []
    stat['time_l'] = []
    stat['time_h'] = []

    stat['delta'] = []
    # чисто чтобы tqdm нормально прогрузился 
    sleep(i / 10)
    print('start', i)

    for p1, p2 in tqdm(pps, desc='find paths', position=i):
        if (p1, p2) in stat:
            continue
        # класический дейкстра
        l, p = None, None
        start = time.time()
        for i in range(NUM_ITERATION):
            p = G.get_shortest_paths(node2id[p1],node2id[p2], weights=G.es["weight"], output="epath",algorithm = 'dijkstra')
            # p = []
            # l, p = nx.single_source_dijkstra(g, p1, p2, weight='length')
        time_l = time.time() - start
        l = 0
        for e in p[0]:
            l += G.es[e]["weight"]
        # иерархический
        h_l, h_p = None, None
        start = time.time()
        for _ in range(NUM_ITERATION):
            # h_l, h_p = nx.single_source_dijkstra(g, p1, p2, weight='length')
            h_l = net_ch.shortest_path_length(node2id[p1], node2id[p2])
            # h_p = []
        time_h = time.time() - start
        # print(l, h_l)
        
        delta = (h_l - l) / l * 100
        stat['l'].append(l)  # длина обычного пути
        stat['h_l'].append(h_l)  # длина иерархического пути
        stat['p'].append(p)  # обычный путь
        stat['h_p'].append(h_p)  # иерархический путь
        stat['delta'].append(delta)  # разница в длине
        stat['time_l'].append(time_l)  # обычное время 
        stat['time_h'].append(time_h)  # иерархическое
    return stat


data = [([p for p in points[i::WORKER]], i) for i in range(WORKER)]
# do_calc(data[0])
with Pool(WORKER) as p:
    res = p.map(do_calc, data)

stat = {}
for l in res:
    for d in l:
        if d not in stat:
            stat[d] = []
        stat[d].extend(l[d])
print('err_mean:', np.mean(stat['delta']))
print('err_min:', np.min(stat['delta']))
print('err_max:', np.max(stat['delta']))
print(np.mean(np.array(stat['time_l']) / np.array(stat['time_h'])))